In [1]:
!rm diff_sorp.h5

rm: cannot remove 'diff_sorp.h5': No such file or directory


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

L = 10.0  # Length of the domain
T = 1.0   # Total simulation time
Nx = 100  # Number of spatial grid points
Nt = 201  # Number of time steps
dx = L / (Nx - 1)  # Spatial step size
dt = T / Nt        # Time step size

for o in range(1000):
    D = np.random.uniform(1, 100)
    Kd = abs(np.random.uniform(D - 10, D + 10)) % 100
    alpha = D / (1 + Kd) * dt / dx**2  # Stability parameter
    print(o, D, Kd)
    if alpha > 0.5:
        print("alpha > 0.5")
        continue
    # Initialize concentration array
    C = np.zeros(Nx)  # Initial concentration
    C[int(Nx / 4):int(Nx / 2)] = 1.0  # Initial condition (step function)

    # Boundary conditions (Dirichlet: fixed concentration at boundaries)
    C[0] = 0.0
    C[-1] = 0.0

    # Store all concentration data over time
    C_all = np.zeros((Nt, Nx))  # Shape: (time steps, spatial points)

    # Time-stepping loop
    for t in range(Nt):
        C_new = C.copy()
        for i in range(1, Nx - 1):
            C_new[i] = C[i] + alpha * (C[i + 1] - 2 * C[i] + C[i - 1])
        C = C_new

        # Apply boundary conditions at each time step
        C[0] = 0.0
        C[-1] = 0.0

        # Store the current concentration profile
        C_all[t, :] = C
    try:
      with h5py.File("diff_sorp.h5", "a") as f:
          g = f.create_group(f"{o}")
          g.create_dataset("D", data=D)
          g.create_dataset("Kd", data=Kd)
          g.create_dataset("data", data=C_all)
    except:
      print(o)

# Now C_all contains the concentration data for all time steps
# You can save it to a file or analyze it further
# np.save("concentration_data.npy", C_all)

0 94.68919560134685 0.33457858778524496
alpha > 0.5
1 74.30630383936519 73.78430892605036
2 94.4407700851652 90.31855441876492
alpha > 0.5
3 99.71362557765698 94.0062461359744
alpha > 0.5
4 98.57524440862478 97.71108272875878
5 75.35207569856256 72.27141087300376
alpha > 0.5
6 76.91499481018847 75.44464211607404
7 15.665210357107782 19.481630549405793
8 20.463450493113655 23.788938407086803
9 49.14740946776755 46.4072656870595
alpha > 0.5
10 50.03434066453582 45.34779686871731
alpha > 0.5
11 89.71444617087289 98.9956742081803
12 98.41467836472013 91.45317195569524
alpha > 0.5
13 24.166169434911023 32.25672056808814
14 80.86708081196335 72.20223988666959
alpha > 0.5
15 76.1877585639998 80.08478708035797
16 52.8762618132468 59.84270479478865
17 87.04614197969846 78.80580954109928
alpha > 0.5
18 68.09217019282187 77.3842297593971
19 32.5762801182906 25.608999217719962
alpha > 0.5
20 45.57489422526605 42.841479197857474
alpha > 0.5
21 11.60361678321315 15.474610606781837
22 42.479443858531

In [3]:
!ls -lh

total 96M
-rw-rw-r-- 1 pes1ug22am100 pes1ug22am100 1.4M Jan 31 13:40 a.gif
-rw-rw-r-- 1 pes1ug22am100 pes1ug22am100  80K Jan 31 13:40 diff-sorp-gen2.ipynb
-rw-rw-r-- 1 pes1ug22am100 pes1ug22am100  75K Jan 31 13:40 diff-sorp-gen.ipynb
-rw-rw-r-- 1 pes1ug22am100 pes1ug22am100  95M Jan 31 14:29 diff_sorp.h5


In [4]:
C_all.shape

(201, 100)

In [5]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import animation
from tqdm import tqdm
import h5py
def visualize_burgers(xcrd, data, path):
    """
    This function animates the Burgers equation

    Args:
    path : path to the desired file
    param: PDE parameter of the data shard to be visualized
    """
    fig, ax = plt.subplots()
    ims = []

    for i in tqdm(range(data.shape[0])):
        if i == 0:
            im = ax.plot(xcrd, data[i].squeeze(), animated=True, color="blue")
        else:
            im = ax.plot(xcrd, data[i].squeeze(), animated=True, color="blue")
        ims.append([im[0]])

    ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True, repeat_delay=1000)

    writer = animation.PillowWriter(fps=15, bitrate=1800)
    ani.save(path, writer=writer)
    plt.close(fig)

In [6]:
visualize_burgers([i for i in range(100)], C_all, "a.gif")

100%|██████████| 201/201 [00:00<00:00, 4930.03it/s]


In [7]:
from scipy.stats import skewnorm

with h5py.File("diff_sorp.h5", "a") as f:
    for key in f.keys():
        u = f[key]["data"][:]
        noise = skewnorm.rvs(a=1, scale=0.2, size=u.shape)
        u_noise = u + noise
        f[key].create_dataset("noise", data=u_noise)